# Supervised Machine Learning: Water Treatment Plant 

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline
sns.set()

## Data Collection

In [3]:
url=f"https://archive.ics.uci.edu/ml/machine-learning-databases/water-treatment/water-treatment.data"

In [4]:
df=pd.read_csv(url, header=None)

In [17]:
names=f"""Date   (Date D/M/YY)
Q-E        (input flow to plant)  
ZN-E       (input Zinc to plant)
PH-E       (input pH to plant) 
DBO-E      (input Biological demand of oxygen to plant) 
DQO-E      (input chemical demand of oxygen to plant)
SS-E       (input suspended solids to plant)  
SSV-E      (input volatile supended solids to plant)
SED-E      (input sediments to plant) 
COND-E     (input conductivity to plant) 
PH-P       (input pH to primary settler)
DBO-P      (input Biological demand of oxygen to primary settler)
SS-P       (input suspended solids to primary settler)
SSV-P      (input volatile supended solids to primary settler)
SED-P      (input sediments to primary settler) 
COND-P     (input conductivity to primary settler)
PH-D       (input pH to secondary settler) 
DBO-D      (input Biological demand of oxygen to secondary settler)
DQO-D      (input chemical demand of oxygen to secondary settler)
SS-D       (input suspended solids to secondary settler)
SSV-D      (input volatile supended solids to secondary settler)
SED-D      (input sediments to secondary settler)  
COND-D     (input conductivity to secondary settler) 
PH-S       (output pH)   
DBO-S      (output Biological demand of oxygen)
DQO-S      (output chemical demand of oxygen)
SS-S       (output suspended solids)
SSV-S      (output volatile supended solids) 
SED-S      (output sediments) 
COND-S     (output conductivity)
RD-DBO-P   (performance input Biological demand of oxygen in primary settler)
RD-SS-P    (performance input suspended solids to primary settler)
RD-SED-P   (performance input sediments to primary settler)
RD-DBO-S   (performance input Biological demand of oxygen to secondary settler)
RD-DQO-S   (performance input chemical demand of oxygen to secondary settler)
RD-DBO-G   (global performance input Biological demand of oxygen)
RD-DQO-G   (global performance input chemical demand of oxygen)
RD-SS-G    (global performance input suspended solids) 
RD-SED-G   (global performance input sediments)"""

In [18]:
names = [[i.replace("-", "_").strip(" ") for i in row.strip(r" )").split(r"  (")] for row in names.split(f"\n")]

In [7]:
df.columns=pd.DataFrame(names)[0]

In [49]:
df.head(20)

,Date,Q_E,ZN_E,PH_E,DBO_E,DQO_E,SS_E,SSV_E,SED_E,COND_E,...,COND_S,RD_DBO_P,RD_SS_P,RD_SED_P,RD_DBO_S,RD_DQO_S,RD_DBO_G,RD_DQO_G,RD_SS_G,RD_SED_G
0,1990-01-03,44101,1.50,7.8,?,407,166,66.3,4.5,2110,...,2000,?,58.8,95.5,?,70.0,?,79.4,87.3,99.6
1,1990-02-03,39024,3.00,7.7,?,443,214,69.2,6.5,2660,...,2590,?,60.7,94.8,?,80.8,?,79.5,92.1,100
2,1990-04-03,32229,5.00,7.6,?,528,186,69.9,3.4,1666,...,1888,?,58.2,95.6,?,52.9,?,75.8,88.7,98.5
3,1990-05-03,35023,3.50,7.9,205,588,192,65.6,4.5,2430,...,1840,33.1,64.2,95.3,87.3,72.3,90.2,82.3,89.6,100
4,1990-06-03,36924,1.50,8.0,242,496,176,64.8,4.0,2110,...,2120,?,62.7,95.6,?,71.0,92.1,78.2,87.5,99.5
5,1990-07-03,38572,3.00,7.8,202,372,186,68.8,4.5,1644,...,1764,?,59.7,96.5,86.7,78.3,90.1,73.1,84.9,100
6,1990-08-03,41115,6.00,7.8,?,552,262,64.1,5.0,1603,...,1703,?,61.9,93.8,89.1,79.8,?,86.2,90.1,99.0
7,1990-09-03,36107,5.00,7.7,215,489,334,40.7,6.0,1613,...,1606,?,70.4,95.6,90.6,53.7,92.1,66.9,94.6,100
8,1990-11-03,29156,2.50,7.7,206,451,194,69.1,4.5,1249,...,1338,46.1,43.6,92.5,85.6,58.2,92.2,73.8,90.2,99.4
9,1990-12-03,39246,2.00,7.8,172,506,200,69.0,5.0,1865,...,1616,21.2,59.7,90.8,88.4,66.1,89.0,69.0,86.5,99.6


## Data Cleaning
### Incorrect Format

In [39]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 527 entries, 0 to 526
Data columns (total 39 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Date      527 non-null    object 
 1   Q_E       527 non-null    object 
 2   ZN_E      527 non-null    object 
 3   PH_E      527 non-null    float64
 4   DBO_E     527 non-null    object 
 5   DQO_E     527 non-null    object 
 6   SS_E      527 non-null    object 
 7   SSV_E     527 non-null    object 
 8   SED_E     527 non-null    object 
 9   COND_E    527 non-null    int64  
 10  PH_P      527 non-null    float64
 11  DBO_P     527 non-null    object 
 12  SS_P      527 non-null    int64  
 13  SSV_P     527 non-null    object 
 14  SED_P     527 non-null    object 
 15  COND_P    527 non-null    int64  
 16  PH_D      527 non-null    float64
 17  DBO_D     527 non-null    object 
 18  DQO_D     527 non-null    object 
 19  SS_D      527 non-null    object 
 20  SSV_D     527 non-null    object

In [43]:
df.Date=df.Date.str.strip(r"D-")

In [46]:
df.Date=pd.to_datetime(df.Date)

### Missing Values

In [38]:
round((df=="?").sum()/df.shape[0], 2)*100

c:\users\nclee\appdata\local\programs\python\python38\lib\site-packages\pandas\core\ops\array_ops.py:253: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  res_values = method(rvalues)


Date         0.0
Q_E          3.0
ZN_E         1.0
PH_E         0.0
DBO_E        4.0
DQO_E        1.0
SS_E         0.0
SSV_E        2.0
SED_E        5.0
COND_E       0.0
PH_P         0.0
DBO_P        8.0
SS_P         0.0
SSV_P        2.0
SED_P        5.0
COND_P       0.0
PH_D         0.0
DBO_D        5.0
DQO_D        2.0
SS_D         0.0
SSV_D        2.0
SED_D        5.0
COND_D       0.0
PH_S         0.0
DBO_S        4.0
DQO_S        3.0
SS_S         1.0
SSV_S        3.0
SED_S        5.0
COND_S       0.0
RD_DBO_P    12.0
RD_SS_P      1.0
RD_SED_P     5.0
RD_DBO_S     8.0
RD_DQO_S     5.0
RD_DBO_G     7.0
RD_DQO_G     5.0
RD_SS_G      2.0
RD_SED_G     6.0
dtype: float64

In [51]:
test=df.drop(columns="RD_DBO_P")

In [52]:
test[(test=="?").sum(axis=1)>0]

c:\users\nclee\appdata\local\programs\python\python38\lib\site-packages\pandas\core\ops\array_ops.py:253: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  res_values = method(rvalues)


,Date,Q_E,ZN_E,PH_E,DBO_E,DQO_E,SS_E,SSV_E,SED_E,COND_E,...,SED_S,COND_S,RD_SS_P,RD_SED_P,RD_DBO_S,RD_DQO_S,RD_DBO_G,RD_DQO_G,RD_SS_G,RD_SED_G
0,1990-01-03,44101,1.50,7.8,?,407,166,66.3,4.5,2110,...,0.02,2000,58.8,95.5,?,70.0,?,79.4,87.3,99.6
1,1990-02-03,39024,3.00,7.7,?,443,214,69.2,6.5,2660,...,0.00,2590,60.7,94.8,?,80.8,?,79.5,92.1,100
2,1990-04-03,32229,5.00,7.6,?,528,186,69.9,3.4,1666,...,0.05,1888,58.2,95.6,?,52.9,?,75.8,88.7,98.5
4,1990-06-03,36924,1.50,8.0,242,496,176,64.8,4.0,2110,...,0.02,2120,62.7,95.6,?,71.0,92.1,78.2,87.5,99.5
5,1990-07-03,38572,3.00,7.8,202,372,186,68.8,4.5,1644,...,0.00,1764,59.7,96.5,86.7,78.3,90.1,73.1,84.9,100
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
509,1991-09-08,30848,0.20,7.7,142,376,144,70.8,3.0,940,...,0.02,947,47.6,97.3,?,77.6,93.7,84.8,90.3,99.3
515,1991-08-18,27527,0.20,7.3,191,240,166,73.5,3.0,1072,...,0.02,999,54.5,88.0,90.0,?,95.8,81.7,93.4,99.3
518,1991-08-21,31914,2.00,7.7,127,274,144,72.2,2.0,1031,...,0.01,1053,?,?,?,77.7,?,?,?,?
521,1991-08-25,29271,0.36,7.5,99,585,140,71.4,?,962,...,?,968,71.6,?,59.0,26.4,74.7,83.8,81.4,?


In [50]:
df[(df=="?").sum(axis=1)>0]

c:\users\nclee\appdata\local\programs\python\python38\lib\site-packages\pandas\core\ops\array_ops.py:253: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  res_values = method(rvalues)


,Date,Q_E,ZN_E,PH_E,DBO_E,DQO_E,SS_E,SSV_E,SED_E,COND_E,...,COND_S,RD_DBO_P,RD_SS_P,RD_SED_P,RD_DBO_S,RD_DQO_S,RD_DBO_G,RD_DQO_G,RD_SS_G,RD_SED_G
0,1990-01-03,44101,1.50,7.8,?,407,166,66.3,4.5,2110,...,2000,?,58.8,95.5,?,70.0,?,79.4,87.3,99.6
1,1990-02-03,39024,3.00,7.7,?,443,214,69.2,6.5,2660,...,2590,?,60.7,94.8,?,80.8,?,79.5,92.1,100
2,1990-04-03,32229,5.00,7.6,?,528,186,69.9,3.4,1666,...,1888,?,58.2,95.6,?,52.9,?,75.8,88.7,98.5
4,1990-06-03,36924,1.50,8.0,242,496,176,64.8,4.0,2110,...,2120,?,62.7,95.6,?,71.0,92.1,78.2,87.5,99.5
5,1990-07-03,38572,3.00,7.8,202,372,186,68.8,4.5,1644,...,1764,?,59.7,96.5,86.7,78.3,90.1,73.1,84.9,100
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
521,1991-08-25,29271,0.36,7.5,99,585,140,71.4,?,962,...,968,40.8,71.6,?,59.0,26.4,74.7,83.8,81.4,?
522,1991-08-26,32723,0.16,7.7,93,252,176,56.8,2.3,894,...,942,?,62.3,93.3,69.8,75.9,79.6,78.6,96.6,99.6
523,1991-08-27,33535,0.32,7.8,192,346,172,68.6,4.0,988,...,950,?,58.3,97.8,83.0,59.1,91.1,74.6,90.7,100
524,1991-08-28,32922,0.30,7.4,139,367,180,64.4,3.0,1060,...,1136,?,65.0,97.1,76.2,66.4,82.0,77.1,88.9,99


In [58]:
df.shape[0]==df.Date.nunique()

True

In [57]:
# a new column to record since first day of record
df["since"]=np.arange(0,df.shape[0], 1)

True